In [40]:
import pandas as pd
import numpy as np

from tensorflow import keras
import tensorflow
from tensorflow.keras.layers import Layer
from keras_tuner.tuners import RandomSearch
import keras_tuner as kt

In [2]:
df = pd.DataFrame()

# avg_open_price = 0

def dataFrame_maker(n,sup):
    global df,total_trades
    lol = pd.read_csv('cm{}AUG2021bhav.csv'.format(n))
    lol = lol[lol['SERIES'] == 'EQ']
    lol = lol[lol['CLOSE'] >= 100]
    lol = lol[lol['TOTALTRADES'] >= 10000]
    lol[n] = ((lol['CLOSE'] - lol['OPEN'])*100 / lol['OPEN']).round(2)
    lol = lol[['SYMBOL',n]]
    
    if sup == 1:
        df = lol.merge(df,on='SYMBOL')
    else:
        df = lol


In [3]:
dataFrame_maker('02',0)

array = ['03','04','05','06','09','10','11','12','13','16','17','18','20','23']

for i in array:
    dataFrame_maker(i,1)

df.dropna(axis=0,inplace=True)
df

,SYMBOL,23,20,18,17,16,13,12,11,10,09,06,05,04,03,02
0,AARTIIND,-2.68,-2.61,-0.29,0.49,-0.13,0.87,0.39,-0.59,-2.99,-1.38,3.16,-2.93,-3.77,2.23,0.23
1,ABFRL,-3.96,-0.47,-1.01,0.17,-0.05,-1.15,0.33,0.90,-1.74,-1.53,-0.54,-1.62,-3.24,-1.25,-1.10
2,ACC,-1.68,-1.70,1.78,0.38,-0.59,0.57,0.60,-0.91,-1.16,-1.51,-1.71,-0.33,-1.41,0.54,1.21
3,ADANIENT,-1.30,-0.55,1.26,0.74,-0.10,-0.55,-0.05,-1.23,-2.79,-3.45,8.06,0.30,-2.09,-1.65,0.22
4,ADANIPORTS,-3.40,-0.41,1.29,-1.86,0.41,-0.04,0.44,0.83,-1.24,-1.97,2.56,-0.21,-3.25,1.01,1.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,VOLTAS,-0.90,-0.74,-0.08,-0.07,1.32,-0.02,-1.89,0.40,-2.42,-2.61,-0.10,0.70,-0.97,-1.77,-1.44
195,WIPRO,0.46,-0.39,-1.00,3.45,-0.33,1.57,0.64,-0.11,0.91,-0.53,-0.95,0.57,-1.11,-0.76,0.69
196,ZEEL,-1.08,-2.10,-1.48,-1.73,-1.25,-1.40,1.26,-1.84,-4.83,0.66,-2.50,-0.88,-2.65,0.64,-0.83
197,ZENSARTECH,-1.20,-2.18,-3.03,3.82,3.82,-0.26,5.31,-1.69,-3.53,-3.52,1.81,-0.32,0.40,0.55,-2.22


# Output Categorisation

In [4]:
def Output_categorization(change):
    if change <=-4.5:
        return "VERY LOW"
    elif change >-4.5 and change <=-2 :
        return "LOW"
    elif change >-2 and change <= 2:
        return "MEDIUM"
    elif change >2 and change<=4.5:
        return "HIGH"
    elif change >4.5:
        return "VERY HIGH"
    
# Change in words
df['output'] = df['23'].apply(Output_categorization)
df = df.drop(['23'], axis = 1)

df.head()

,SYMBOL,20,18,17,16,13,12,11,10,09,06,05,04,03,02,output
0,AARTIIND,-2.61,-0.29,0.49,-0.13,0.87,0.39,-0.59,-2.99,-1.38,3.16,-2.93,-3.77,2.23,0.23,LOW
1,ABFRL,-0.47,-1.01,0.17,-0.05,-1.15,0.33,0.90,-1.74,-1.53,-0.54,-1.62,-3.24,-1.25,-1.10,LOW
2,ACC,-1.70,1.78,0.38,-0.59,0.57,0.60,-0.91,-1.16,-1.51,-1.71,-0.33,-1.41,0.54,1.21,MEDIUM
3,ADANIENT,-0.55,1.26,0.74,-0.10,-0.55,-0.05,-1.23,-2.79,-3.45,8.06,0.30,-2.09,-1.65,0.22,MEDIUM
4,ADANIPORTS,-0.41,1.29,-1.86,0.41,-0.04,0.44,0.83,-1.24,-1.97,2.56,-0.21,-3.25,1.01,1.47,LOW


In [13]:
X = df.iloc[:,1:-1]
y = df['output']
y.value_counts()

MEDIUM      99
LOW         64
VERY LOW    30
HIGH         6
Name: output, dtype: int64

In [14]:
y = pd.get_dummies(y,columns=['y'])
y

,HIGH,LOW,MEDIUM,VERY LOW
0,0,1,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0
...,...,...,...,...
194,0,0,1,0
195,0,0,1,0
196,0,0,1,0
197,0,0,1,0


In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# ANN

In [35]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense,LeakyReLU,PReLU,ELU,Dropout,Activation,Embedding,Flatten,BatchNormalization
from keras.activations import relu,sigmoid

# Initialising the ANN
classifier = Sequential()

# output_dim or unit is for output layer ; kernel_initializer or init
# Adding the input layer and the first hidden layer
classifier.add(Dense( units= 6, kernel_initializer = 'he_uniform',activation='relu',input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(units = 4, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'Adamax', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)



Epoch 1/100
10/10 [==============================] - 1s 19ms/step - loss: 1.3725 - accuracy: 0.0645 - val_loss: 1.2431 - val_accuracy: 0.1087
Epoch 2/100
10/10 [==============================] - 0s 4ms/step - loss: 1.3210 - accuracy: 0.0753 - val_loss: 1.2117 - val_accuracy: 0.1304
Epoch 3/100
10/10 [==============================] - 0s 5ms/step - loss: 1.2829 - accuracy: 0.0860 - val_loss: 1.1834 - val_accuracy: 0.1304
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 1.2493 - accuracy: 0.0968 - val_loss: 1.1574 - val_accuracy: 0.1304
Epoch 5/100
10/10 [==============================] - 0s 5ms/step - loss: 1.2176 - accuracy: 0.0968 - val_loss: 1.1336 - val_accuracy: 0.1304
Epoch 6/100
10/10 [==============================] - 0s 5ms/step - loss: 1.1888 - accuracy: 0.1075 - val_loss: 1.1108 - val_accuracy: 0.1304
Epoch 7/100
10/10 [==============================] - 0s 5ms/step - loss: 1.1615 - accuracy: 0.1075 - val_loss: 1.0896 - val_accuracy: 0.1304
Epoch 8/100


Epoch 59/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6498 - accuracy: 0.3656 - val_loss: 0.6816 - val_accuracy: 0.3478
Epoch 60/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6461 - accuracy: 0.3656 - val_loss: 0.6786 - val_accuracy: 0.3478
Epoch 61/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6425 - accuracy: 0.3763 - val_loss: 0.6757 - val_accuracy: 0.3478
Epoch 62/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6389 - accuracy: 0.3656 - val_loss: 0.6727 - val_accuracy: 0.3478
Epoch 63/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6355 - accuracy: 0.3656 - val_loss: 0.6700 - val_accuracy: 0.3478
Epoch 64/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6321 - accuracy: 0.3656 - val_loss: 0.6672 - val_accuracy: 0.3478
Epoch 65/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6289 - accuracy: 0.3656 - val_loss: 0.6645 - val_accuracy: 0.3478
Epoch 

In [45]:


def build_model(hp):
    model = keras.Sequential()
    
    # the neural network can have layers between 2 to 20 
    for i in range(hp.Int('num_layers',2,20)):
        model.add(keras.layers.Dense(
            # neurons will be chaning between 32 and 512 , where its added 32 ea
            units = hp.Int('units_'+str(i),min_value=32,max_value=512,step=32),
            activation = 'relu'
        ))
        model.add(keras.layers.Dense(4,activation='Softmax'))         # output layer , acitvation linear as it regressions 
        model.compile(
            # Random search will select from this learning rates
            optimizer = keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
            loss = keras.losses.CategoricalCrossentropy(),
#             loss= keras.losses.categorical_crossentropy(from_logits=True),
            metrics=['accuracy']
        )
    
    return model
     

In [46]:
tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial =3,
    directory = 'project3',
)

In [47]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [48]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)

tuner.search(
    X_train,
    y_train,
    epochs = 10,
    validation_data = (X_test,y_test)
)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.5055555502573649

Best val_accuracy So Far: 0.5055555502573649
Total elapsed time: 00h 00m 28s
INFO:tensorflow:Oracle triggered exit


In [50]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps

In [52]:
print(best_hps.get('num_layers'))
print(best_hps.get('learning_rate'))
print(best_hps.get('units_1'))
print(best_hps.get('units_2'))
print(best_hps.get('units_3'))
print(best_hps.get('units_4'))

4
0.01
128
416
512
32


In [54]:
# Initialising the ANN
classifier = Sequential()

# output_dim or unit is for output layer ; kernel_initializer or init
# Adding the input layer and the first hidden layer
classifier.add(Dense( units= 128, kernel_initializer = 'he_uniform',activation='relu',input_dim = 14))

# Adding the second hidden layer
classifier.add(Dense(units = 416, kernel_initializer = 'he_uniform',activation='relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 512, kernel_initializer = 'he_uniform',activation='relu'))

# Adding the second hidden layer
classifier.add(Dense(units = 32, kernel_initializer = 'he_uniform',activation='relu'))
# Adding the output layer
classifier.add(Dense(units = 4, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = keras.optimizers.Adam(0.01), loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
model_history=classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 10, epochs = 100)


Epoch 1/100
10/10 [==============================] - 1s 20ms/step - loss: 1.8345 - accuracy: 0.3226 - val_loss: 0.5560 - val_accuracy: 0.1957
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5241 - accuracy: 0.3763 - val_loss: 0.5134 - val_accuracy: 0.4130
Epoch 3/100
10/10 [==============================] - 0s 7ms/step - loss: 0.5019 - accuracy: 0.4946 - val_loss: 0.5190 - val_accuracy: 0.4565
Epoch 4/100
10/10 [==============================] - 0s 6ms/step - loss: 0.4550 - accuracy: 0.5591 - val_loss: 0.6165 - val_accuracy: 0.4130
Epoch 5/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4243 - accuracy: 0.6344 - val_loss: 0.5191 - val_accuracy: 0.5000
Epoch 6/100
10/10 [==============================] - 0s 6ms/step - loss: 0.3866 - accuracy: 0.6667 - val_loss: 0.5797 - val_accuracy: 0.4130
Epoch 7/100
10/10 [==============================] - 0s 6ms/step - loss: 0.3483 - accuracy: 0.6667 - val_loss: 0.6815 - val_accuracy: 0.3913
Epoch 8/100


Epoch 59/100
10/10 [==============================] - 0s 6ms/step - loss: 0.1322 - accuracy: 0.9892 - val_loss: 6.4062 - val_accuracy: 0.3478
Epoch 60/100
10/10 [==============================] - 0s 6ms/step - loss: 0.2345 - accuracy: 0.8925 - val_loss: 4.8331 - val_accuracy: 0.4348
Epoch 61/100
10/10 [==============================] - 0s 7ms/step - loss: 0.9110 - accuracy: 0.7849 - val_loss: 2.8695 - val_accuracy: 0.2609
Epoch 62/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3653 - accuracy: 0.6452 - val_loss: 0.8998 - val_accuracy: 0.5000
Epoch 63/100
10/10 [==============================] - 0s 7ms/step - loss: 0.4399 - accuracy: 0.6344 - val_loss: 2.7252 - val_accuracy: 0.3696
Epoch 64/100
10/10 [==============================] - 0s 6ms/step - loss: 0.4993 - accuracy: 0.7312 - val_loss: 0.9038 - val_accuracy: 0.2609
Epoch 65/100
10/10 [==============================] - 0s 7ms/step - loss: 0.3695 - accuracy: 0.6989 - val_loss: 1.1692 - val_accuracy: 0.3478
Epoch 

In [ ]:
lol.to_csv(r'jeez bredth1.csv', index = False, header=True)